### Imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pytz
import duckdb
import time

import requests as rq
import json
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import os
from dotenv import load_dotenv
from pathlib import Path

import asyncio
import json
import websockets
from urllib.parse import urlparse, parse_qs
import numpy as np

### Run simulation

#### Get token list

In [4]:
DB_PATH_1 = str(Path.cwd() / "token_list.duckdb")
dbw_1 = duckdb.connect(DB_PATH_1)

tokens = dbw_1.execute("SELECT token_address FROM token_list").fetchall()
token_list = [t[0] for t in tokens]

#### Read from database

In [9]:
# tokens_sql = ", ".join(f"'{t}'" for t in token_list)

# DB_PATH = str(Path.cwd() / "price_ws_stream.duckdb")
# dbr = duckdb.connect(DB_PATH)

# df = dbr.execute(f"""
#     SELECT *
#     FROM price_stream
#     WHERE token_address IN ({tokens_sql})
#     QUALIFY row_number() OVER (
#         PARTITION BY token_address
#         ORDER BY last_updated_at DESC
#     ) = 1
# """).df()

# df

In [ ]:
# def trade_status(usd_price, entry):

#     if usd_price > entry * 1.2:
#         return "take_profit"
#     if usd_price < entry * 0.8:
#         return "stop_loss"
    
#     return "monitoring"

# # Per-token state
# state = {t: {"last_ts": None, "entry_price": None, "last_status": None} for t in token_list}

# # Use ANSI escape codes in the print strings for colors
# COLORS = {
#     "monitoring": "\033[34m",  # blue
#     "stop_loss": "\033[31m",   # red
#     "take_profit": "\033[32m", # green
# }
# RESET = "\033[0m"

# dbr = duckdb.connect(DB_PATH)

# while True:
#     time.sleep(2)

#     for token in token_list:
#         last_ts = state[token]["last_ts"]
#         entry_price = state[token]["entry_price"]
#         last_status = state[token]["last_status"]

#         if last_ts is None:
#             query = f"""
#             SELECT * FROM price_stream
#             WHERE token_address = '{token}'
#             ORDER BY last_updated_at ASC
#             """
#         else:
#             query = f"""
#             SELECT * FROM price_stream
#             WHERE token_address = '{token}'
#               AND last_updated_at > '{last_ts}'
#             ORDER BY last_updated_at ASC
#             """

#         df = dbr.execute(query).df()
#         if df.empty:
#             continue

#         if entry_price is None:
#             entry_price = df.iloc[0]["usd_price"]

#         for _, row in df.iterrows():
#             status = trade_status(row["usd_price"], entry_price)

#             price_fmt = f"{row['usd_price']:.8f}"
#             entry_fmt = f"{entry_price:.8f}"

#             if status != last_status:
#                 line = f"[{row['last_updated_at']}] {token} {status} @ {price_fmt} (entry = {entry_fmt} USD)"
#                 print(f"{COLORS.get(status, '')}{line}{RESET}")
#                 last_status = status

#             last_ts = row["last_updated_at"]

#         state[token].update({"last_ts": last_ts, "entry_price": entry_price, "last_status": last_status})